In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

In [ ]:
import pandas as pd
import numpy as np

def process_excel(filepath, seq_len=100, step=1, verbose=False):
    df = pd.read_excel(filepath, header=None)
    
    if df.shape[0] <= 1 or df.shape[1] <= 1:
        raise ValueError(f"文件 {filepath} 行/列过少: {df.shape}")
    
    df = df.iloc[1:, 1:].reset_index(drop=True)
    
    depth_col_indices = [3 + 4*i + 1 for i in range(6)]
    max_needed_col = max(depth_col_indices)
    if max_needed_col >= df.shape[1]:
        raise ValueError(
            f"文件 {filepath} 列数不足，期望至少 {max_needed_col+1} 列（当前 {df.shape[1]} 列）。"
            " 请确认 Excel 列的排列是否与预期一致。"
        )

    downstream_level = df.iloc[0, 0]
    try:
        downstream_level = float(downstream_level)
    except Exception:
        downstream_level = np.nan

    flows = df.iloc[:, 1:3].astype(float).values  # shape (T, 2)

    depths = df.iloc[:, depth_col_indices].astype(float).values  # shape (T, 6)

    T = len(df)
    X_seq_list = []
    X_static_list = []
    Y_list = []

    start_idx = seq_len * step
    if T <= start_idx:
        if verbose:
            print(f"Warning: 文件 {filepath} 时间步 (T={T}) <= seq_len*step ({start_idx})，没有样本。")
        return (np.empty((0, seq_len, flows.shape[1]), dtype=np.float32),
                np.empty((0, 1), dtype=np.float32),
                np.empty((0, depths.shape[1]), dtype=np.float32))

    for t in range(start_idx, T, step):
        seq_input = flows[t - seq_len*step : t : step, :]   
        if seq_input.shape[0] != seq_len: 
            if verbose:
                print(f"跳过 t={t}：序列长度 {seq_input.shape[0]} != seq_len {seq_len}")
            continue

        # 静态输入：仅下游边界水位（定值）
        static_input = np.array([downstream_level], dtype=np.float32)  # shape (1,)

        # 输出：当前时间步的 6 点深度
        y = depths[t]  # shape (6,)

        # 若任一处含 NaN，跳过该样本（可选，也可以改为插值/填充）
        if np.isnan(seq_input).any() or np.isnan(static_input).any() or np.isnan(y).any():
            if verbose:
                print(f"跳过 t={t}：发现 NaN (seq_input 或 static_input 或 y).")
            continue

        X_seq_list.append(seq_input.astype(np.float32))
        X_static_list.append(static_input.astype(np.float32))
        Y_list.append(y.astype(np.float32))

    if len(X_seq_list) == 0:
        # 无有效样本，返回空数组（保持维度）
        return (np.empty((0, seq_len, flows.shape[1]), dtype=np.float32),
                np.empty((0, 1), dtype=np.float32),
                np.empty((0, depths.shape[1]), dtype=np.float32))

    X_seq_arr = np.stack(X_seq_list, axis=0)     # (N, seq_len, 2)
    X_static_arr = np.stack(X_static_list, axis=0)  # (N, 1)
    Y_arr = np.stack(Y_list, axis=0)             # (N, 6)

    return X_seq_arr, X_static_arr, Y_arr


In [ ]:
# Excel 文件目录
data_dir = r"D:\0DATA"   # 修改为你的存放路径
file_pathes = []
for i in range(1, 49):  # 1 ~ 48
    file_path = os.path.join(data_dir, f"case{i}.xlsx")
    file_pathes.append((i, file_path))

# 手动指定测试集工况索引（这里举例选择13个，可以自己改）
test_indices = [22, 24, 3, 6, 15, 31, 34, 18, 38, 39, 42, 43, 48] 
train_indices = [i for i in range(1, (len(file_pathes) + 1)) if i not in test_indices]

print("训练集工况数:", len(train_indices))
print(train_indices)
print("测试集工况数:", len(test_indices))
print(test_indices)

print(type(file_pathes))
for path in file_pathes:
    print(path[0], path[1])

In [ ]:
import torch

X_seq_train, X_static_train, Y_train = [], [], []
X_seq_test, X_static_test, Y_test = [], [], []

test_data_per_case = {}  # 保存每个工况的测试数据

for idx, filepath in file_pathes:
    # filepath = os.path.join(data_dir, fname)
    print(f"处理工况 {idx}: {filepath}")

    X_seq, X_static, Y = process_excel(filepath)
    if(torch.isnan(torch.from_numpy(X_seq)).any()):
        print(idx, "  |  ", filepath)

    if idx in train_indices:
        X_seq_train.append(X_seq)
        X_static_train.append(X_static)
        Y_train.append(Y)
    else:
        X_seq_test.append(X_seq)
        X_static_test.append(X_static)
        Y_test.append(Y)
        test_data_per_case[idx] = (X_seq, X_static, Y)

# 拼接训练、测试数据
X_seq_train = np.concatenate(X_seq_train, axis=0)
X_static_train = np.concatenate(X_static_train, axis=0)
Y_train = np.concatenate(Y_train, axis=0)

X_seq_test = np.concatenate(X_seq_test, axis=0)
X_static_test = np.concatenate(X_static_test, axis=0)
Y_test = np.concatenate(Y_test, axis=0)

print("训练集输入 shape:", X_seq_train.shape, X_static_train.shape)
print("测试集输入 shape:", X_seq_test.shape, X_static_test.shape)
print("训练集输出 shape:", Y_train.shape)
print("测试集输出 shape:", Y_test.shape)


In [ ]:
output_dir = r"D:\0DATA\DataSet\processed_data"
os.makedirs(output_dir, exist_ok=True)

# 合并后的训练/测试集
np.save(os.path.join(output_dir, "X_seq_train.npy"), X_seq_train)
np.save(os.path.join(output_dir, "X_static_train.npy"), X_static_train)
np.save(os.path.join(output_dir, "Y_train.npy"), Y_train)

np.save(os.path.join(output_dir, "X_seq_test.npy"), X_seq_test)
np.save(os.path.join(output_dir, "X_static_test.npy"), X_static_test)
np.save(os.path.join(output_dir, "Y_test.npy"), Y_test)

# 测试集按工况单独保存
case_dir = os.path.join(output_dir, "test_cases")
os.makedirs(case_dir, exist_ok=True)

for idx, (X_seq, X_static, Y) in test_data_per_case.items():
    np.save(os.path.join(case_dir, f"X_seq_case{idx}.npy"), X_seq)
    np.save(os.path.join(case_dir, f"X_static_case{idx}.npy"), X_static)
    np.save(os.path.join(case_dir, f"Y_case{idx}.npy"), Y)

print("✅ 数据保存完成！")


✅ 数据保存完成！


In [6]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
import pandas as pd
import gc

# 设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用设备:", device)

# ------- PatchTST 配置 -------
class Cfg:
    pass

configs = Cfg()
configs.enc_in = 3         # 2 原始 + 1 静态特征
configs.seq_len = 100
configs.pred_len = 1
configs.e_layers = 3
configs.n_heads = 4
configs.d_model = 64 #64
configs.d_ff = 128 # 128
configs.dropout = 0.4
configs.fc_dropout = 0.4
configs.head_dropout = 0.4
configs.patch_len = 16
configs.stride = 8
configs.padding_patch = None
configs.revin = True
configs.affine = True
configs.subtract_last = False
configs.decomposition = True #False
configs.kernel_size = 25
configs.individual = False


使用设备: cuda


In [ ]:
# 损失函数
# 六个点坐标
coords = torch.tensor([
    [532043.125, 3401273.750],
    [532036.375, 3401250.250],
    [532028.938, 3401220.750],
    [532246.000, 3401357.500],
    [532248.438, 3401325.500],
    [532248.313, 3401293.000]
], dtype=torch.float32)

##NOTE 相邻位置之间的水深不应该相差太大
neighbor_pairs = [(0,1), (1,2), (3,4), (4,5)]
distances = torch.tensor([torch.norm(coords[i]-coords[j]).item() for i,j in neighbor_pairs], dtype=torch.float32)

def continuity_loss(pred, y, beta=0.1, device="cpu"):
    mse = nn.MSELoss()(pred, y)
    spatial_loss = 0.0
    for k, (i,j) in enumerate(neighbor_pairs):
        dij = distances[k].to(device)
        spatial_loss += torch.mean(((pred[:,i]-pred[:,j])**2)/(dij**2))
    return mse + beta*spatial_loss


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PatchTST import Model as PatchTSTModel   

# ========= 图残差模块 =========
class GraphResidualBlock(nn.Module):
    def __init__(self, num_nodes, hidden_dim=32):
        """
        num_nodes: 节点数 (例如6)
        hidden_dim: 中间隐藏维度
        """
        super().__init__()
        self.num_nodes = num_nodes

        # 可学习邻接矩阵 [N, N]
        self.A_param = nn.Parameter(torch.randn(num_nodes, num_nodes))

        # 节点特征变换
        self.fc1 = nn.Linear(1, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        """
        x: [B, N] 或 [B, 1, N]
        """
        if x.dim() == 3:  # [B, 1, N]
            x = x.squeeze(1)  # [B, N]

        A = torch.softmax(self.A_param, dim=-1)  # [N, N]

        h = x.unsqueeze(-1)         # [B, N, 1]
        h = self.fc1(h)             # [B, N, H]
        h = torch.matmul(A, h)      # [B, N, H]
        h = F.relu(h)
        h = self.fc2(h).squeeze(-1) # [B, N]

        return x + h  # 残差连接

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- 子模块 ---
class GRN(nn.Module):
    """Gated Residual Network"""
    def __init__(self, input_dim, hidden_dim, output_dim=None, dropout=0.1):
        super().__init__()
        output_dim = output_dim or input_dim
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.gate = nn.Linear(output_dim, output_dim)
        self.norm = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x
        x = F.gelu(self.fc1(x))
        x = self.dropout(self.fc2(x))
        gate = torch.sigmoid(self.gate(x))
        x = gate * x + (1 - gate) * residual
        return self.norm(x)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]


In [ ]:
# --- CTFT ---
class CompactTFT(nn.Module):
    def __init__(self, enc_in, d_model=64, n_heads=4, n_layers=2, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(enc_in, d_model)
        self.pos_encoding = PositionalEncoding(d_model)

        # 层堆叠
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_heads,
            dim_feedforward=d_model*2, dropout=dropout, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.static_fuse = GRN(d_model, d_model*2, d_model, dropout=dropout)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, static_emb=None):
        x = self.input_proj(x)
        x = self.pos_encoding(x)
        x = self.encoder(x)

        if static_emb is not None:
            s = static_emb.unsqueeze(1).repeat(1, x.size(1), 1)
            x = self.static_fuse(x + s)

        return self.norm(x)

In [ ]:
from PatchTST import Model as PatchTSTModel   # 根据实际路径

# ------------------------------
# PatchTST 回归器 (TFT 在前)
# ------------------------------
class PatchTSTRegressor(nn.Module):
    def __init__(self, configs, c_out=6, gnn_hidden=32, d_model=64):
        super().__init__()
        self.pred_len = configs.pred_len
        self.orig_c_in = configs.enc_in    
        self.c_out = c_out
        
        self.tft = CompactTFT(enc_in=self.orig_c_in, d_model=d_model, n_heads=4, n_layers=2, dropout=0.3)

        import copy
        new_configs = copy.deepcopy(configs)
        new_configs.enc_in = d_model    
        self.patch_model = PatchTSTModel(new_configs, max_seq_len=1024)
        self.graph_block = GraphResidualBlock(num_nodes=c_out, hidden_dim=gnn_hidden)

        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.pred_len * d_model, 32),   
            nn.ReLU(),
            nn.Linear(32, c_out)
        )

    def forward(self, x):
        x = self.tft(x)  # [B, seq_len, d_model]
        out = self.patch_model(x)  # [B, pred_len, d_model]
        if out.shape[1] != self.pred_len:
            out = out[:, -self.pred_len:, :]

        yhat = self.head(out)  # [B, c_out]
        # 图残差块 refine
        yhat = self.graph_block(yhat)  # [B, c_out]

        return yhat


# ------------------------------
# 初始化模型
# ------------------------------
model = PatchTSTRegressor(configs, c_out=6, gnn_hidden=16, d_model=32).to(device)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

B = 4
seq_len = configs.seq_len   
c_in = configs.enc_in      
c_out = 6

dummy_x_seq = torch.randn(B, seq_len, 2).to(device)       
dummy_x_static = torch.randn(B, 1).to(device)             
dummy_x = torch.cat([dummy_x_seq, dummy_x_static.unsqueeze(1).expand(-1, seq_len, -1)], dim=-1)  

model = PatchTSTRegressor(configs, c_out=c_out).to(device)
model.eval()

with torch.no_grad():
    out = model(dummy_x)
    print("模型输出 shape:", out.shape)  # 期望 [B, c_out]，例如 [4, 6]


模型输出 shape: torch.Size([4, 6])


In [ ]:
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

data_dir = r"D:\0DATA\DataSet\processed_data"

X_seq_train = np.load(f"{data_dir}/X_seq_train.npy")
X_static_train = np.load(f"{data_dir}/X_static_train.npy")
Y_train = np.load(f"{data_dir}/Y_train.npy")

X_seq_test = np.load(f"{data_dir}/X_seq_test.npy")
X_static_test = np.load(f"{data_dir}/X_static_test.npy")
Y_test = np.load(f"{data_dir}/Y_test.npy")

print("训练集:", X_seq_train.shape, X_static_train.shape, Y_train.shape)
print("测试集:", X_seq_test.shape, X_static_test.shape, Y_test.shape)

# ========== Min-Max 归一化 ==========
def minmax_scale(train, test):
    min_val = train.min(axis=0, keepdims=True)
    max_val = train.max(axis=0, keepdims=True)
    train_norm = (train - min_val) / (max_val - min_val + 1e-8)
    test_norm = (test - min_val) / (max_val - min_val + 1e-8)
    return train_norm, test_norm, min_val, max_val

# 对三类数据分别做归一化
X_seq_train, X_seq_test, X_seq_min, X_seq_max = minmax_scale(
    X_seq_train.reshape(-1, X_seq_train.shape[-1]),
    X_seq_test.reshape(-1, X_seq_test.shape[-1])
)
X_seq_train = X_seq_train.reshape(-1, configs.seq_len, 2)
X_seq_test = X_seq_test.reshape(-1, configs.seq_len, 2)

X_static_train, X_static_test, X_static_min, X_static_max = minmax_scale(X_static_train, X_static_test)
Y_train, Y_test, Y_min, Y_max = minmax_scale(Y_train, Y_test)

# numpy -> torch
X_seq_train_t = torch.tensor(X_seq_train, dtype=torch.float32)
X_static_train_t = torch.tensor(X_static_train, dtype=torch.float32)
Y_train_t = torch.tensor(Y_train, dtype=torch.float32)

X_seq_test_t = torch.tensor(X_seq_test, dtype=torch.float32)
X_static_test_t = torch.tensor(X_static_test, dtype=torch.float32)
Y_test_t = torch.tensor(Y_test, dtype=torch.float32)

# 拼接时间序列 + 静态特征 (最后一维)
train_X = torch.cat([X_seq_train_t, X_static_train_t.unsqueeze(1).expand(-1, configs.seq_len, -1)], dim=-1)
test_X = torch.cat([X_seq_test_t, X_static_test_t.unsqueeze(1).expand(-1, configs.seq_len, -1)], dim=-1)

# TensorDataset + DataLoader
BATCH_SIZE = 512
train_dataset = TensorDataset(train_X, Y_train_t)
val_dataset = TensorDataset(test_X, Y_test_t)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=10, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8, pin_memory=True)



### 读取各个工况的测试集数据 ================================================
import os
import glob

# ===== 读取各工况测试集 =====
case_dir = os.path.join(data_dir, "test_cases")
test_data_per_case = {}

# 找出所有 case 的 X_seq 文件
case_files = glob.glob(os.path.join(case_dir, "X_seq_case*.npy"))

for f in case_files:
    # 提取工况编号
    fname = os.path.basename(f)  # eg: X_seq_case12.npy
    case_id = int(fname.replace("X_seq_case", "").replace(".npy", ""))

    # 对应的 X_static 和 Y 文件
    X_seq_case = np.load(os.path.join(case_dir, f"X_seq_case{case_id}.npy"))
    X_static_case = np.load(os.path.join(case_dir, f"X_static_case{case_id}.npy"))
    Y_case = np.load(os.path.join(case_dir, f"Y_case{case_id}.npy"))

    test_data_per_case[case_id] = (X_seq_case, X_static_case, Y_case)

print(f"✅ 已加载 {len(test_data_per_case)} 个工况测试集")


训练集: (55085, 100, 2) (55085, 1) (55085, 6)
测试集: (20793, 100, 2) (20793, 1) (20793, 6)
✅ 已加载 13 个工况测试集


In [15]:
# Cell: 安全 AMP 训练 + 评估函数

from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import torch
import numpy as np

def train_one_epoch(model, loader, optimizer, scaler, device):
    model.train()
    total_loss, total_rmse, total_mape, total_r2 = 0,0,0,0
    n_samples = 0

    for xb, yb in loader:
        B = xb.size(0)
        xb = xb.to(device, dtype=torch.float32)
        yb = yb.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        # 自动混合精度
        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            out = model(xb)   # [B, 6]
            loss = continuity_loss(out, yb, beta=0.1, device=device)

        if torch.isnan(loss):
            print("⚠️ Loss is NaN, skipping this batch.")
            continue

        # 反向 + 梯度裁剪 + AMP
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)  # ✅ 解锁梯度
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # 可选，防梯度爆炸
        scaler.step(optimizer)
        scaler.update()

        # CPU 指标计算
        y_np, out_np = yb.detach().cpu().numpy(), out.detach().cpu().numpy()

        if np.isnan(out_np).any() or np.isnan(y_np).any():
            print("⚠️ NaN detected in out_np or y_np, skipping this batch.")
            continue
        if np.isinf(out_np).any():
            print("⚠️ Inf detected in out_np, clipping.")
            out_np = np.nan_to_num(out_np, nan=0.0, posinf=1e6, neginf=-1e6)

        rmse = np.sqrt(mean_squared_error(y_np, out_np))
        #mape = mean_absolute_percentage_error(y_np, out_np)
        r2 = r2_score(y_np, out_np)

        total_loss += loss.item() * B
        total_rmse += rmse * B
        #total_mape += mape * B
        total_r2 += r2 * B
        n_samples += B

    return total_loss/n_samples, total_rmse/n_samples, 0, total_r2/n_samples


def evaluate(model, loader, device):
    model.eval()
    total_loss, total_rmse, total_mape, total_max, total_r2 = 0,0,0,0,0
    n_samples = 0

    with torch.no_grad():
        for xb, yb in loader:
            B = xb.size(0)
            xb = xb.to(device, dtype=torch.float32)
            yb = yb.to(device, dtype=torch.float32)

            out = model(xb)
            loss = continuity_loss(out, yb, beta=0.1, device=device)

            y_np, out_np = yb.detach().cpu().numpy(), out.detach().cpu().numpy()
            rmse = np.sqrt(mean_squared_error(y_np, out_np))
            mape = mean_absolute_percentage_error(y_np, out_np)
            max_err = np.max(np.abs(y_np - out_np))
            #r2 = r2_score(y_np, out_np)

            total_loss += loss.item() * B
            total_rmse += rmse * B
            total_mape += mape * B
            total_max += max_err * B
            #total_r2 += r2 * B
            n_samples += B

    return total_loss/n_samples, total_rmse/n_samples, total_mape/n_samples, total_max/n_samples, 0 #total_r2/n_samples


In [16]:
print(model)

from torchinfo import summary

summary(
    model,
    input_data=torch.randn(2, configs.seq_len, configs.enc_in).to(device),  # 对应 PatchTSTRegressor.forward
    verbose=1
)

PatchTSTRegressor(
  (tft): CompactTFT(
    (input_proj): Linear(in_features=3, out_features=64, bias=True)
    (pos_encoding): PositionalEncoding()
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
          )
          (linear1): Linear(in_features=64, out_features=128, bias=True)
          (dropout): Dropout(p=0.3, inplace=False)
          (linear2): Linear(in_features=128, out_features=64, bias=True)
          (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.3, inplace=False)
          (dropout2): Dropout(p=0.3, inplace=False)
        )
      )
    )
    (static_fuse): GRN(
      (fc1): Linear(in_features=64, out_features=128, bias=True)
      (fc2): Linear(in_features

Layer (type:depth-idx)                                                      Output Shape              Param #
PatchTSTRegressor                                                           [2, 6]                    --
├─CompactTFT: 1-1                                                           [2, 100, 64]              20,864
│    └─Linear: 2-1                                                          [2, 100, 64]              256
│    └─PositionalEncoding: 2-2                                              [2, 100, 64]              --
│    └─TransformerEncoder: 2-3                                              [2, 100, 64]              --
│    │    └─ModuleList: 3-1                                                 --                        66,944
│    └─LayerNorm: 2-4                                                       [2, 100, 64]              128
├─Model: 1-2                                                                [2, 1, 64]                --
│    └─series_decomp: 2-5               

In [ ]:
epochs = 5000
best_val_loss = float('inf')
best_val_me = float("inf")

train_loss_list, val_loss_list = [], []

save_path = r"D:\0DATA\TFT_PatchTST_GR"
os.makedirs(save_path, exist_ok=True)
excel_file = os.path.join(save_path, "train_val_loss_1008.xlsx")

optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scaler = torch.amp.GradScaler()

for epoch in range(1, epochs+1):
    train_loss, train_rmse, train_mape, train_r2 = train_one_epoch(model, train_loader, optimizer, scaler, device)
    gc.collect()
    torch.cuda.empty_cache()
    val_loss, val_rmse, val_mape, val_max, val_r2 = evaluate(model, val_loader, device)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model, os.path.join(save_path, f"best_val_mse_1013_test.pth"))

    if val_max < best_val_me:
        best_val_me = val_max
        torch.save(model, os.path.join(save_path, f"best_val_max_1013_test.pth"))


    # --- 输出指标 ---
    print(
        f"Epoch {epoch:02d}/{epochs} | "
        f"Train Loss: {train_loss:.6f} , Val Loss: {val_loss:.6f} | "
        f"Train RMSE: {train_rmse:.6f} , Train R2: {train_r2:.6f} | "
        f"Val RMSE: {val_rmse:.6f} , Val MAPE: {val_mape:.6f} , Val Max_Err: {val_max:.6f}"
    )

    # ===== 记录 loss =====
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)

# ===== 保存到 Excel =====
df_loss = pd.DataFrame({
    "Train Loss": train_loss_list,
    "Validation Loss": val_loss_list
})
df_loss.to_excel(excel_file, index_label="Epoch")

print("✅ 训练完成，最佳验证集 Loss:", best_val_loss)
print(f"训练/验证 Loss 已保存至: {excel_file}")
